# Setup Postgres Database on Ubuntu VM

Let us understand how to setup Postgres on Ubuntu VM. We will use Docker to set it up.

* Setup Postgres using Docker
* Accessing Postgres using Docker CLI
* Create Database and User
* Executing SQL Scripts
* Setup SQL Workbench
* Connect to SQL Workbench - Postgres
* SQL Workbench Features
* Troubleshooting Issues

## Setup Postgres using Docker
As we have successfully setup Docker on our VM, let us go ahead and setup Postgres Database.

Here are the steps involved in setting up Postgres Database using Docker:
* Pull postgres image
* Create container for postgres
* Start the container
* Review the logs to ensure that container is created with out any issues

Here are important commands to manage images and containers:

* Managing images - `docker image`

|Command           |Description |
|------------------|------------|
|docker image pull |Pull image  |
|docker image rm   |Remove image|
|docker image build|Build image |
* Managing containers - `docker container`

|Command                |Description                       |
|-----------------------|----------------------------------|
|docker container create|Create container                  |
|docker container start |Start container                   |
|docker container run   |Build, Create and Start container |
|docker container logs  |Check logs of docker container    |
|docker container rm    |Remove stopped container          |
|docker container rm -f |Stop and Remove running container |
|docker container ls    |List containers                   |

> For most of these commonly used commands we have alternative with out image or container as keyword in the command - for example we can say `docker rm` to remove the container and `docker rmi` to remove the image.

```
docker pull postgres

docker container create \
  --name sms_pg \
  -p 5433:5432 \
  -h sms_pg \
  -e POSTGRES_PASSWORD=itversity \
  postgres
  
docker container start sms_pg

docker container logs sms_pg
docker container logs -f sms_pg
```

## Accessing Postgres using Docker CLI

As the container is created, let us understand how to connect to Postgres Database using Docker CLI.

* We can use `docker container exec` or `docker exec` to connect to the container.
* You can attach to the container by running `bash` using `docker exec`.
* Also you can run single commands with out attaching the container - example: `docker exec -it sms_pg hostname -f`

> You have to use terminal to run these commands

* Attach to sms_db container - `docker exec -it sms_pg bash`
* Run command to get hostname - `hostname -f`
* Run command to connect to Postgres Database - `psql -U postgres`
* You can also directly connect to Postgres Database using
```
docker exec -it sms_pg psql -U postgres
```
* Use `\q` to come out of the Postgres CLI.

## Create Database and User

Let us create **database** and **user** using `psql` CLI.

* Postgres is multi tenant database.
* We typically follow these steps to create a database which can be used by connecting as specific user.
  * Connect to postgres server as user postgres (super user)
  * Create database - `sms_db`
  * Create user with password - `sms_user`
  * Grant permissions on database to user
  
```
docker exec -it sms_pg psql -U postgres

CREATE DATABASE sms_db;
CREATE USER sms_user WITH ENCRYPTED PASSWORD 'sms_password';
GRANT ALL ON DATABASE sms_db TO sms_user;

\l --to list databases
\q --to quit from postgres CLI
```

* Make sure to validate by connecting using sms_user.
> When we use psql directly with in the container, you might be able to connect to database even with out password. Don't worry about it for now.

> Connect to postgres using newly created user

```
docker exec -it sms_pg psql -U sms_user -d sms_db -W

SELECT current_database();
CREATE TABLE t (i INT);
INSERT INTO t VALUES (1);
SELECT * FROM t;

\d
\d t

DROP TABLE t;
```



## Executing SQL Scripts

Let us understand how to execute SQL Scripts using psql. We will be creating new database and then running the script which is downloaded from GitHub..

* Copy the script to sms_pg container.
* Make sure database is created.
* Run the script to create tables and insert data.
* Validate that tables are created and data is inserted.

**Clone Repository**

```
# Make sure you are in the home directory in the host (not in the container)
cd # to be in home directory
git clone https://www.github.com/dgadiraju/retail_db.git
```

**Copy Script and Validate**

```
docker container cp retail_db sms_pg:/

docker exec -it sms_pg ls -ltr /retail_db

docker exec -it sms_pg psql -U postgres
```

**Create Database and User for retail_db**

```
docker exec -it sms_pg psql -U postgres

CREATE DATABASE retail_db;
CREATE USER retail_user WITH ENCRYPTED PASSWORD 'retail_password';
GRANT ALL ON DATABASE retail_db TO retail_user;
```

**Create tables and copy data**

We will be running script to create tables and copy data.

```
docker exec -it sms_pg psql -U retail_user -d retail_db -W

\i /retail_db/create_db_tables_pg.sql

\i /retail_db/load_db_tables_pg.sql
```

**Validate - Run Queries**

Make sure you are in right database and run these queries.

```
docker exec -it sms_pg psql -U retail_user -d retail_db -W

\d

\d orders

SELECT * FROM orders LIMIT 10;

SELECT count(1) FROM orders;
```

## Setup SQL Workbench

Let us understand how to setup and use SQL Workbench.

**Why SQL Workbench**

Let us see the details why we might have to use SQL Workbench.
* Using Database CLIs such psql for postgres, mysql etc can be cumbersome for those who are not comfortable with command line interfaces.
* Database IDEs such as SQL Workbench will provide required features to run queries against databases with out worrying to much about underlying data dictionaries.
* SQL Workbench provide required features to review databases and objects with out writing queries or running database specific commands.
* Also Database IDEs provide capabilities to preserve the scripts we develop.
> **In short Database IDEs such as SQL Workbench improves productivity.**

**Alternative IDEs**

There are several IDEs in the market.
* TOAD
* SQL Developer for Oracle
* MySQL Workbench
and many others

**Install Workbench**

Here are the instructions to setup SQL Workbench.
* Download SQL Workbench (typically zip file)
* Unzip and launch

Once installed we need to perform below steps which will be covered in detail as part of next topic.
* Download JDBC driver for the database we would like to connect.
* Get the database connectivity information and connect to the database.

## Connect to SQL Workbench - Postgres

Let us connect to Postgres Database using SQL Workbench.

* We are trying to connect to Postgres Database that is running as part of Docker container running in a Ubuntu 18.04 VM provisioned from GCP.
* We have published Postgres database port to port 5433 on Ubuntu 18.04 VM.
* We typically use ODBC or JDBC to connect to a Database from remote machines (our PC).
* Here are the pre-requisites to connect to a Database on GCP.
  * Make sure 5433 port is opened as part of the firewalls.
  * If you have telnet configured on your system on which SQL Workbench is installed, make sure to validate by running telnet command using ip or DNS Alias and port number 5433.
  * Ensure that you have downloaded right JDBC Driver for Postgres.
  * Make sure to have right credentials (username and password).
  * Ensure that you have database created on which the user have permissions.
* You can validate credentials and permissions to the database by installing postgres client on Ubuntu 18.04 VM and then by connecting to the database using the credentials.
* Once you have all the information required along with JDBC jar, ensure to save the information as part of the profile. You can also validate before saving the details by using **Test** option.

## SQL Workbench Features

Here are some of the key features, you have to familiar with related to SQL Workbench.
* Saving profiles to connect to multiple databases.
* Develop SQL files and preserve them for future usage.
* Access data dictionary or information schema to validate tables, columns, sequences, indexes, constraints etc.
* Generate scripts out of existing data.
* Ability to manage database objects with out writing any commands. We can drop tables, indexes, sequences etc by right clicking and then dropping.

Almost all leading IDEs provide all these features in similar fashion.

**Usage Scenarios**

Here are **some of the usage scenarios** for database IDEs such as SQL Workbench as part of day to day responsibilities.
* Developers for generating and validating data as part of unit testing.
* Testers to validate data for their test cases.
* Business Analysts and Data Analysts to run ad hoc queries to understand the data better.
* Developers to troubleshoot data related production issues using read only accounts.

## Troubleshooting Issues

Let us understand how to troubleshoot common database connectivity issues related to Postgres DB.

* We need to have telnet or nc on the machine from which we are trying to connect to database to troubleshoot the issues.
* Host not reachable - Root causes
  * Typo in the host
  * Using Private ip instead of public ip
  * Might be blocked by the firewall
* Unable to authenticate - Root causes
  * Incorrect username or password
  * Incorrect Hostname or Port Number
  * Insufficient privileges for the user over the database
  * Incorrect Database Name
* Here are the following things you need to keep in mind while connecting to database running in docker container created in Ubuntu 18.04 Virtual Machine using GCP.
```
docker container create \
  --name sms_pg \
  -p 5433:5432 \
  -h sms_pg \
  -e POSTGRES_PASSWORD=itversity \
  postgres
```
  * Make sure port number used by Postgres database in the container is published to host. In our example it is 5433. Port 5432 in container is published to port 5433 on the host.
  * Ensure that port is opened in network firewall for the Virtual Machine in GCP using GCP Web Console.
  * If you have `telnet` on your machine from which you are trying to connect on VM in GCP, run `telnet` command by passing public ip of GCP VM and the port 5433.